<a href="https://colab.research.google.com/github/EilieYoun/Narnia-Edu/blob/main/Lecture/240813_snu/03_3D_Field_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울대 예측 AI 실습 : 3D Field Predict


* 날짜:
* 이름:


## 학습내용
```
- 3D 데이터셋에 대해 이해하고 적절한 DataLoader를 구성 한다.
- 3D Field 예측 문제에 적합한 모델을 구성하고, 학습을 진행한다.
```

## **(0) 환경세팅**
---

### **| 라이브러리 설치**

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install torchio monai

### **| 데이터 압축 풀기**

In [ ]:
!unzip bracket_field.zip -d ./bracket_field

### **| Utils**

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import pandas as pd
from mpl_toolkits.mplot3d.art3d import Poly3DCollection


def plot_voxel(voxel_grid):
    fig = plt.figure(figsize=(5, 5))

    ax1 = fig.add_subplot(111, projection='3d')
    ax1.voxels(voxel_grid, edgecolor='k')
    ax1.set_title('Voxel visualization')

    plt.show()
    plt.close()

def plot_voxel_grid(grid, mask, alpha=0.7):
    grid_shape = grid.shape

    colors = plt.cm.rainbow(grid)
    colors[..., 3] = alpha  # alpha 채널 추가

    fig = plt.figure(figsize=(18, 6))

    ax1 = fig.add_subplot(111, projection='3d')
    ax1.voxels(mask, facecolors=colors, edgecolor='none')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    cb1 = fig.colorbar(plt.cm.ScalarMappable(cmap='rainbow', norm=plt.Normalize(vmin=0, vmax=1.1)), ax=ax1, shrink=0.5)
    cb1.set_label('Value')

    plt.show()
    plt.close()

# 3차원데이터의 위에서 본 모습만 2d plot으로 그리기 -> 굉장히 빠르게 시각화 진행가능
def plot_topview(xs, alpha=0.8, save_path=None, rows=1, mn=None, mx=None):
    n = len(xs)
    cols = (n + rows - 1) // rows  # Calculate the number of columns needed

    fig, axs = plt.subplots(rows, cols, figsize=(3*cols, 3*rows))
    axs = np.array(axs).reshape(rows, cols)  # Ensure axs is a 2D array

    for i, (ax, x) in enumerate(zip(axs.flatten(), xs)):
        top_view = np.max(x, axis=2)[::-1]

        cax1 = ax.imshow(top_view, cmap='rainbow', alpha=alpha, origin='lower', vmin=mn, vmax=mx)
        cb1 = fig.colorbar(cax1, ax=ax, shrink=0.75)
        cb1.set_label('Value')

    for ax in axs.flatten()[n:]:
        ax.axis('off')  # Turn off axes for any extra subplots

    if save_path is not None:
        plt.savefig(save_path)
    else:
        plt.show()

    plt.close()

## **(1) Dataset**

### **| EDA**

**데이터 소개**

`Simulated Jet Engine Bracket Dataset (SimJEB)` 는 다양한 용도로 사용 가능한 공개 데이터셋입니다. 이 데이터셋은 기하학적 처리 및 대체 모델링 알고리즘 테스트, 구조 설계 전략 연구 등에 활용될 수 있습니다. 데이터셋은 2013년 "GE 제트 엔진 브래킷 챌린지"의 디자인을 기반으로 하며, 정리, 방향 설정, 스케일 조정, 메싱, 시뮬레이션 과정을 거친 디자인을 포함하고 있습니다​

이번 시간에 다룰 데이터는 `SimJEB` STL 32x32x32 해상도의 데이터의 각 지점에서의 변위를 측정한 것입니다. 변위는 구조 해석, 지진 공학, 재료 과학 등 다양한 분야에서 중요한 역할을 합니다. 예를 들어, 구조물의 변위를 측정함으로써 구조물의 안정성을 평가하고, 지진 시 구조물의 거동을 분석할 수 있습니다.  변위가 작으면 구조물의 손상 가능성이 줄어들고 구조적 안정성을 확보할 수 있습니다.

In [ ]:
import glob

paths = sorted(glob.glob('./bracket_field/*.npy'))
print(len(paths))

**데이터 시각화**

In [ ]:
idx = 32
data = np.load(paths[idx])
print('data shape: ', data.shape)

plot_voxel(data) # 0 또는 1로만 되어있는거 처럼 보이지만 실제로는 그렇지 않다.

# plot_voxel_grid: 모든 값을 값에 따라 다른 색깔로 표현
plot_voxel_grid(data, data!=0) # data가 0이 아닌 영역만 그리기

In [ ]:
_=plt.hist(data.flatten()) # 0인 부분은 빈공간, 그외 부분은 0 ~ 1.09~ 정도의 변위값을 지니고 있다.

### **| DataLoader**

간략하게 데이터 구성을 확인했으니 이를 적절한 `DataLoader` 클래스를 구축하겠습니다. `DataLoader`는 아래와 같은 기능이 포함되어야 합니다.

```
- __init__ : 데이터셋 초기화.
    - field_paths :
    - batch_size: 8
    - shuffle: True
    - dtype: torch.float32
    - dtype: 데이터 유형 (torch.float32 기본값).

- __len__ : 데이터셋의 전체 길이를 반환.

- __getitem__ : 인덱스에 해당하는 데이터 항목을 반환.
    - DataFrame을 로드하여 필요한 데이터 불러오기
    - 성능 변수를 적절하게 정규화
    - 전처리된 인풋 텐서 (x)와 타깃 텐서 (y) 반환
    
- get_loader : 데이터 로더를 반환.

- get_batch : 지정된 인덱스의 배치를 반환.
```


In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader


class BracketFieldProcess(Dataset):

  def __init__(self,
               field_paths,
               batch_size=8,
               shuffle=True,
               dtype=torch.float32
               ):

    self.field_paths = field_paths
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.dtype = dtype

  def __len__(self):
    return len(self.field_paths)

  def __getitem__(self, idx):
    field_path = self.field_paths[idx]
    field = np.load(field_path) # 32,32,32
    voxel = np.where(field==0, 0, 1) # 32,32,32

    mask = (field!=0)  # bkg아닌 변위 값이 존재하는 곳
    field[mask] = np.log(field[mask]) * -1. / 10. # 변위 값을 log 변형

    voxel = torch.from_numpy(voxel).type(self.dtype).unsqueeze(0) # 1, 32, 32, 32
    field = torch.from_numpy(field).type(self.dtype).unsqueeze(0) # 1, 32, 32, 32

    return voxel, field



  def get_loader(self):
    loader = DataLoader(self, batch_size=self.batch_size, shuffle=self.shuffle, pin_memory=True)
    return loader

  def get_batch(self, idx=0):
    ds = self.get_loader()
    for i, batch in enumerate(ds):
        if i == idx : break
    return batch

**데이터 분할**

In [ ]:
from sklearn.model_selection import train_test_split

train_paths, test_paths = train_test_split(paths, test_size=0.1, random_state=42)
print(len(paths), len(train_paths), len(test_paths))

**Processor 구축**

In [ ]:
bsz = 8
train_pp = BracketFieldProcess(train_paths, batch_size=bsz, shuffle=True)
test_pp = BracketFieldProcess(train_paths, batch_size=bsz, shuffle=False)

**Batch 데이터 확인**

In [ ]:
train_xs, train_ys = train_pp.get_batch(0)
print('- Train batch shape: ', train_xs.shape, train_ys.shape)
plot_topview(train_xs[:,0].numpy()) # n, 32, 32, 32 numpy array
plot_topview(train_ys[:,0].numpy())

test_xs, test_ys = test_pp.get_batch(0)
print('- Test batch shape: ', test_xs.shape, test_ys.shape)
plot_topview(test_xs[:,0].numpy())
plot_topview(test_ys[:,0].numpy())

In [ ]:
y  = train_ys.numpy().flatten()
mask = (y!=0)  # bkg아닌 변위 값이 존재하는 곳

y[mask] = np.log(y[mask]) * -1. / 10.
_=plt.hist(y[mask], bins=50)
_=plt.hist(y, bins=50)


## **(2) 모델**
---

* 이미지 출처 : (https://github.com/AghdamAmir/3D-UNet)

<img src="https://github.com/AghdamAmir/3D-UNet/raw/main/3D-UNET.png" alt="UNet" width="700"/>

이번 시간에는 `monai.networks.nets` 의 `UNet` 이용해 `3D Unet`을 구축하겠습니다. 이 모델은 1, 32, 32, 32 인풋을 받아 다시 1, 32, 32, 32 크기의 아웃풋을 내놓는 모델이 됩니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from monai.networks.nets import UNet

class UNet3D(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet3D, self).__init__()
        self.unet = UNet(
            spatial_dims = 3,
            in_channels = in_channels,
            out_channels = out_channels,
            channels = (16, 32, 64, 128, 256),
            strides = (2,2,2,2),
            num_res_units = 2,
        )

    def forward(self, x):
        return self.unet(x)

In [ ]:
# 모델 구조 생성
structure = UNet3D()

# 모델구조 테스트
# 입력 데이터
inputs = torch.randn(1, 1, 32, 32, 32) # (n, 1, 32, 32, 32)
outputs = structure(inputs)
print(outputs.shape)

### **| 모델 구축**

`PyTorch Lightning`을 사용하여 훈련, 검증, 최적화 루틴을 포함하는 `BracketFieldPredictor`를 정의합니다.

```
- __init__ : 예측 모델(predictor)을 초기화합니다.
- forward : 입력 데이터를 예측 모델에 전달하여 출력을 반환합니다.
- configure_optimizers : Adam 옵티마이저와 코사인 조정 학습률 스케줄러를 설정합니다.
- training_step : 훈련 배치에서 입력 데이터와 타깃을 받아 모델 출력을 계산하고, MSE 손실을 계산하여 로깅합니다.
- validation_step : 검증 배치에서 입력 데이터와 타깃을 받아 모델 출력을 계산하고, MSE 손실을 계산하여 로깅합니다.
- fit : 훈련 및 검증 데이터를 사용하여 모델을 훈련합니다. 조기 종료와 체크포인트 저장 기능을 포함합니다.
- test_step : 테스트 배치에서 입력 데이터와 타깃을 받아 모델 출력을 계산하고, R² 점수와 MAE를 포함한 성능 지표를 로깅합니다.
- test : 테스트 데이터를 사용하여 모델 성능을 평가합니다.
- infer : 주어진 데이터 로더에서 예측을 수행하고, 필요 시 스케일러를 사용하여 예측값을 역변환합니다.
```

In [ ]:
import torch.nn as nn
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import  CSVLogger

from sklearn.metrics import r2_score, mean_absolute_error



class BracketFieldPredictor(pl.LightningModule):
    def __init__(self, structure, *args, **kwargs):
        super().__init__()
        self.structure = structure

    def forward(self, x):
        x = self.structure(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.init_lr)
        scheduler = CosineAnnealingLR(optimizer, T_max=self.epochs)
        return [optimizer], [{"scheduler": scheduler, "interval": "epoch"}]

    def training_step(self, batch, batch_idx):
        data, target = batch
        output = self(data)
        loss = nn.MSELoss()(output, target)
        lr = self.optimizers().param_groups[0]['lr']

        self.log('train_loss', loss)
        self.log('learning_rate', lr, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        data, target = batch
        output = self(data)
        loss =  nn.MSELoss()(output, target)
        lr = self.optimizers().param_groups[0]['lr']

        self.log('valid_loss', loss)
        return loss

    def fit(self, train_loader, save_dir,  valid_loader=None, init_lr=1e-3, epochs=10, patience=5, infer_ds=None):
        self.init_lr = init_lr
        self.epochs = epochs
        self.infer_ds = infer_ds
        self.save_dir = save_dir

        # valid loss 기준으로 최적 모델 저장하기
        if valid_loader is not None:
            monitor = 'valid_loss'
        else:
            monitor = 'train_loss'

        checkpoint_callback = ModelCheckpoint(
            dirpath=save_dir,
            filename='ckp_model',
            save_top_k=1,
            verbose=True,
            monitor=monitor,
            mode='min'
        )

        # log
        csv_logger = CSVLogger(save_dir, name="csv_logs")

        # train
        self.trainer = Trainer(
            accelerator='cuda',
            max_epochs=epochs,
            default_root_dir=save_dir,
            callbacks=[checkpoint_callback],
            logger=[csv_logger],
            log_every_n_steps=len(train_loader)
        )

        self.trainer.fit(self, train_dataloaders=train_loader, val_dataloaders=valid_loader)


    def test_step(self, batch, batch_idx):
        data, target = batch
        output = self(data)
        loss = nn.MSELoss()(output, target)
        self.log('test_loss', loss)

    def test(self, data_loader, device='cuda'):
        self.trainer = Trainer(
            accelerator=device,
        )
        results = self.trainer.test(self, dataloaders=data_loader)
        return results

    def on_train_epoch_end(self): # lightning 모듈에서 이미 정의되어있는 함수. (1epoch 끝날때마다 자동 실행)
        # 모델이 잘 학습되고 있는지 시각화를 해보자

        # inference 에 필요한 데이터
        if self.infer_ds is not None:
          imgs  = []
          xs, ys = self.infer_ds
          preds = self(xs.cuda())

          # numpy 로 변경, 차원 변경(n, 1, 32, 32, 32) -> (n, 32, 32, 32)
          xs = xs.numpy()[:,0]
          ys = ys.numpy()[:,0]
          preds = preds.detach().cpu().numpy()[:,0]

          imgs.extend(xs)
          imgs.extend(ys)
          imgs.extend(preds)
          # imgs = [x1, x2, x3, ... y1, y2, y3 ... pred1, pred2, ...]

          epoch = self.trainer.current_epoch
          plot_topview(
              np.array(imgs),
              save_path = f'{self.save_dir}/sample_e{epoch:05d}.png',
              rows=3, # 이미지 3줄로 출력
          )

### | **학습 및 평가**

**모델 인스턴스**

In [ ]:
model = BracketFieldPredictor(structure=structure)

**학습**

In [ ]:
train_loader = train_pp.get_loader()
test_loader = test_pp.get_loader()
test_xs, test_ys = test_pp.get_batch(0)
model.fit(train_loader, save_dir='unet3d_1', valid_loader = test_loader, epochs=100, infer_ds = [test_xs, test_ys], init_lr=1e-3)

**테스트**

In [ ]:
model = BracketFieldPredictor.load_from_checkpoint('./unet3d_1/ckp_model.ckpt', structure=structure)
model.test(test_loader)

**결과 시각화**

In [ ]:
preds = model(test_xs)
preds = preds.detach().cpu().numpy()[:,0]
reals = test_ys.numpy()[:,0]
diffs = np.abs(reals - preds)

plot_topview(reals)
plot_topview(preds)
plot_topview(diffs, mn=0, mx=1)

In [ ]:
idx = 0
plot_voxel_grid(reals[idx], reals[idx]!=0)
plot_voxel_grid(preds[idx], reals[idx]!=0) # 모델이 예측한 preds 의 배경은 preds 값 기준이 아닌 인풋 데이터 기준으로 배경을 설정
plot_voxel_grid(diffs[idx], reals[idx]!=0, alpha=0.4)